<a href="https://colab.research.google.com/github/AForbis/Bootcamp_Final_Project/blob/datasourcing/final_project_segment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd 
import requests
import io
import csv

In [2]:
#eventually we will replace this with a database
url = 'SeasonData/AllNBA_Unpivoted.xlsx'
#load in All NBA teams, drop the league column (we are not including the ABA, I already deleted all that from the file)
All_NBA_Unpivoted_df = pd.DataFrame(pd.read_excel(url)).drop(['Lg'],axis=1)
#rename the players column
All_NBA_Unpivoted_df=All_NBA_Unpivoted_df.rename(columns = {'Value':'PlayerName'})
#create column with the start year
All_NBA_Unpivoted_df['Season_Start_Year'] = All_NBA_Unpivoted_df['Season'].apply(lambda row: row[0:4])

All_NBA_Unpivoted_df.head(10)

,Season,Tm,Position,PlayerName,Season_Start_Year
0,2019-20,1st,C,Anthony Davis,2019
1,2019-20,1st,F,Giannis Antetokounmpo,2019
2,2019-20,1st,F,LeBron James,2019
3,2019-20,1st,G,James Harden,2019
4,2019-20,1st,G,Luka Dončić,2019
5,2019-20,2nd,C,Nikola Jokić,2019
6,2019-20,2nd,F,Kawhi Leonard,2019
7,2019-20,2nd,F,Pascal Siakam,2019
8,2019-20,2nd,G,Damian Lillard,2019
9,2019-20,2nd,G,Chris Paul,2019


In [3]:
#cleaning up the names with crazy characters
All_NBA_Unpivoted_df['PlayerName_Formatted'] = All_NBA_Unpivoted_df['PlayerName'].apply(lambda row: row.replace("ć","c"))
All_NBA_Unpivoted_df['PlayerName_Formatted'] = All_NBA_Unpivoted_df['PlayerName_Formatted'].apply(lambda row: row.replace("č","c"))
All_NBA_Unpivoted_df['PlayerName_Formatted'] = All_NBA_Unpivoted_df['PlayerName_Formatted'].apply(lambda row: row.replace("ž","z"))

All_NBA_Unpivoted_df.head(10)

,Season,Tm,Position,PlayerName,Season_Start_Year,PlayerName_Formatted
0,2019-20,1st,C,Anthony Davis,2019,Anthony Davis
1,2019-20,1st,F,Giannis Antetokounmpo,2019,Giannis Antetokounmpo
2,2019-20,1st,F,LeBron James,2019,LeBron James
3,2019-20,1st,G,James Harden,2019,James Harden
4,2019-20,1st,G,Luka Dončić,2019,Luka Doncic
5,2019-20,2nd,C,Nikola Jokić,2019,Nikola Jokic
6,2019-20,2nd,F,Kawhi Leonard,2019,Kawhi Leonard
7,2019-20,2nd,F,Pascal Siakam,2019,Pascal Siakam
8,2019-20,2nd,G,Damian Lillard,2019,Damian Lillard
9,2019-20,2nd,G,Chris Paul,2019,Chris Paul


In [4]:
#get list of all of the players that have made an all nba team
AllNBAPlayersList = All_NBA_Unpivoted_df['PlayerName_Formatted'].unique()
AllNBAPlayersList

array(['Anthony Davis', 'Giannis Antetokounmpo', 'LeBron James',
       'James Harden', 'Luka Doncic', 'Nikola Jokic', 'Kawhi Leonard',
       'Pascal Siakam', 'Damian Lillard', 'Chris Paul', 'Rudy Gobert',
       'Jayson Tatum', 'Jimmy Butler', 'Ben Simmons', 'Russell Westbrook',
       'Paul George', 'Stephen Curry', 'Joel Embiid', 'Kevin Durant',
       'Kyrie Irving', 'Blake Griffin', 'Kemba Walker',
       'LaMarcus Aldridge', 'DeMar DeRozan', 'Karl-Anthony Towns',
       'Victor Oladipo', 'Isaiah Thomas', 'DeAndre Jordan',
       'Draymond Green', 'John Wall', 'DeMarcus Cousins',
       'Andre Drummond', 'Klay Thompson', 'Kyle Lowry', 'Marc Gasol',
       'Pau Gasol', 'Tim Duncan', 'Joakim Noah', 'Dwight Howard',
       'Kevin Love', 'Tony Parker', 'Al Jefferson', 'Goran Dragic',
       'Kobe Bryant', 'Carmelo Anthony', 'David Lee', 'Dwyane Wade',
       'Andrew Bynum', 'Tyson Chandler', 'Dirk Nowitzki', 'Rajon Rondo',
       'Derrick Rose', "Amar'e Stoudemire", 'Al Horford', 'Za

In [5]:
url = 'SeasonData/Seasons_Stats.xlsx'
#load in season stats
season_stats_df = pd.DataFrame(pd.read_excel(url))
season_stats_df['Player'] = season_stats_df['Player'].apply(lambda row: row.replace("*",""))
season_stats_df.head()

,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,2017,A.J. Hammons,C,24,DAL,22,0.0,163,8.4,0.472,...,0.450,8,28,36,4,1,13,10,21,48
1,2017,Aaron Brooks,PG,32,IND,65,0.0,894,9.5,0.507,...,0.800,18,51,69,125,25,9,66,93,322
2,2017,Aaron Gordon,SF,21,ORL,80,72.0,2298,14.4,0.530,...,0.719,116,289,405,150,64,40,89,172,1019
3,2017,Aaron Harrison,SG,22,CHO,5,0.0,17,-2.2,0.102,...,0.500,0,3,3,3,0,0,0,2,1
4,2017,Adreian Payne,PF,25,MIN,18,0.0,135,14.4,0.505,...,0.737,9,24,33,7,8,7,8,32,63


In [6]:
# get list of every nba player
AllPlayersList = season_stats_df.Player.unique()
AllPlayersList_df = pd.DataFrame(AllPlayersList)
AllPlayersList_df.to_csv('listofplayers_test.csv')

In [7]:
#check and make sure every all-nba player shows up in the season data dataset
#players who weren't in the league in 2017 will show up here, that's okay!
#I cleaned up a bunch of names in the excel
for player in AllNBAPlayersList:
    if player not in AllPlayersList:   
        print(player)

Luka Doncic
Jayson Tatum
Ben Simmons
